In [1]:
Folder1 = "test/txt"
Folder2 = "train/beth/txt"
Folder3 = "train/partners/txt"

In [2]:
import os
import re
import time
import requests
import pandas as pd

In [3]:
BlackList = ["YES", "NO", "NORMAL", "REPORT", "OTHER", "SIZE", "TIME", "DATE", "DOCTOR", "TIME", "PLAN", "FOLLOW", "MEDICINE", "REVIEW", "COUNT"]

# Functions -start-

In [4]:
def GetAllFilesListInFolder(f):
    files = os.listdir(f)
    textFiles = []
    
    for file in files:
        if file.endswith(".txt"):
            textFiles.append(os.path.join(f, file))
    return textFiles


def ReadAllFilesInList(Files):
    content = []
    for file in Files:
        with open(file, "r", encoding='utf-8-sig') as f:
            content.append(f.read())
    return content

def FindAbbreviations(text):
    # (?=[A-Z0-9]*[A-Z])
    # İfadenin devamında sıfır veya daha fazla "büyük harf ve rakam" ve ardından en az bir büyük harf olmalıdır.
    #
    # [A-Z0-9]{2,}
    # En az bir büyük harf ve rakamdan oluşan, en az 2 karakterlik veri
    #
    # Kombinasyonlu ifade de bu ifadelerin arasına "/" karakteri gelirse de kabul ediyor.
    # DM2, PCRE, DM2/A (kombinasyonlu)
    abbreviation_pattern = r'\b(?=[A-Z0-9]*[A-Z])[A-Z0-9]{2,6}\b'
    combined_pattern = r'\b(?=[A-Z0-9]*[A-Z])[A-Z0-9]{2,6}/(?=[A-Z0-9]*[A-Z])[A-Z0-9]{1,5}\b'

    tokens = re.findall(abbreviation_pattern, text)
    tokens.extend(re.findall(combined_pattern, text))

    abbreviations = {token for token in tokens if re.search(r'[A-Z]', token) and token not in BlackList and not token[0].isdigit()}

    return sorted(abbreviations)

def FilterAbbreviationsWithTreshold(allAbbreviations, threshold):
    # Çok fazla kısaltma bulunurken, bunların bazıları yanlış olabilir.
    # Az sayıda tekrar eden veri varsa bunları sileceğiz, belirli bir eşik değerin üstündeki
    # tekrar sayıları o verinin potansiyel kısaltma olduğunu ifade edeceğini kabul ediyoruz.
    abbreviationsCount = {}

    for abbreviation in allAbbreviations:
        abbreviationsCount[abbreviation] = abbreviationsCount.get(abbreviation, 0) + 1

    return [abbr for abbr, count in abbreviationsCount.items() if count > threshold]

# Functions -end-

In [5]:
Files1 = GetAllFilesListInFolder(Folder1)
Files2 = GetAllFilesListInFolder(Folder2)
Files3 = GetAllFilesListInFolder(Folder3)

Content1 = ReadAllFilesInList(Files1)
Content2 = ReadAllFilesInList(Files2)
Content3 = ReadAllFilesInList(Files3)

In [6]:
AllAbbreviations = []

for Contents in [Content1, Content2, Content3]:
    for Content in Contents:
        arr = FindAbbreviations(Content)
        AllAbbreviations.extend(arr)

AllAbbreviations = FilterAbbreviationsWithTreshold(AllAbbreviations, 3)
AllAbbreviations = set(AllAbbreviations)
print(AllAbbreviations)

{'RESUME', 'III', 'NT', 'ADMIT', 'ALK', 'SL', 'RSV', 'PLT', 'CAR', 'BY', 'VSS', 'FVC', 'ASA', 'VIT', 'T4', 'CERVIX', 'GEN', 'PAST', 'NEG', 'H2', 'NGTD', 'CH', 'UA', 'EKG', 'ARTERY', 'HR', 'DIS', 'OMH', 'OF', 'MITRAL', 'SOCIAL', 'FRI', 'NON', 'P0', 'YC', 'LDH', 'VS', 'NONE', 'P1', 'UTERUS', 'LENNI', 'PMH', 'BXH', 'OM1', 'ANEMIA', 'V4', 'EDC', 'EF', 'PCA', 'ECHO', 'ND', 'VQ', 'T2', 'TYPE', 'MOUTH', 'OT', 'SEEN', 'CANCER', 'PACU', 'LA', 'ECG', 'PRN', 'EMS', 'NKA', 'NIACIN', 'EOS', 'OM', 'MEALS', 'S2', 'INR', 'HCTZ', 'ED', 'PEEP', 'FOR', 'OBS', 'CLINIC', 'MG', 'RECTAL', 'ENT', 'CCU', 'TRAUMA', 'SVR', 'CVP', 'T1', 'CNS', 'STAGE', 'BASOS', 'HCL', 'DNR', 'HIV', 'XII', 'WED', 'A1', 'OMR', 'RATE', 'LVEF', 'O2', 'LI', 'BYPASS', 'URINE', 'HEAD', 'ABG', 'CMC', 'MD', 'DS', 'CRI', 'STEMI', 'TWO', 'SC', 'ASSIST', 'HO', 'BB', 'SQ', 'PTA', 'BP', 'RIGHT', 'TP', 'STAT', 'NPH', 'TIMES', 'WITH', 'ICU', 'THIS', 'USED', 'SYSTEM', 'REASON', 'OM2', 'PARKS', 'HLGMC', 'EMR', 'OU', 'USOH', 'RETURN', 'MRA', 'RV', 

In [7]:
TRFile = "correction_1900.xlsx"
TRDatas = pd.read_excel(TRFile, sheet_name=None)
try:
    TRDatas = TRDatas["Sheet1"]
except:
    pass

In [8]:
TRDatas

,Index,Content,Notes,Content.1
0,En(1),# AV fistula : pt with h/o numerous clots in A...,AV:arteriovenous:arteriyovenöz,# AV fistula : pt with h/o numerous clots in A...
1,Tr_second(1),# AV fistül: AV fistülde çok sayıda pıhtı öyk...,NaN,# AV fistül: AV fistülde çok sayıda pıhtı öyk...
2,Corrected(1),# AV fistül : AV fistülde çok sayıda pıhtı öyk...,NaN,NaN
3,NaN,NaN,NaN,NaN
4,En(2),The patient had coarse rhonchi on examination .,NaN,The patient had coarse rhonchi on examination .
...,...,...,...,...
7594,Corrected(1899),Agresif kan ürünü replasmanına rağmen hasta 12...,NaN,NaN
7595,NaN,NaN,NaN,NaN
7596,En(1900),He had undergone C-spine surgery for spinal st...,NaN,He had undergone C-spine surgery for spinal st...
7597,Tr_second(1900),Sol üst ekstremitede rezidüel spastik parezi v...,NaN,Sol üst ekstremitede rezidüel spastik parezi v...


In [9]:
TRDatas = TRDatas.dropna(subset=["Notes"])
TRDatas

,Index,Content,Notes,Content.1
0,En(1),# AV fistula : pt with h/o numerous clots in A...,AV:arteriovenous:arteriyovenöz,# AV fistula : pt with h/o numerous clots in A...
12,En(4),#. Pleural effusions : Likely due to CHF altho...,CHF: congestive heart failure:KKY: Konjestif k...,#. Pleural effusions : Likely due to CHF altho...
13,Tr_second(4),#. Plevral efüzyonlar : dış merkeze kabulünde ...,Admission kelimesinin hem hastaneye yatış hem ...,#. Plevral efüzyonlar : dış merkeze kabulünde ...
20,En(6),"Atrioseptal defect repair in 1970 , cardiomyop...",CPAP: Continuous Positive Airway Pressure: Dev...,"Atrioseptal defect repair in 1970 , cardiomyop..."
24,En(7),69 y/o female wtih PMH significant for ESRD on...,PMH:Past Medical History: özgeçmiş; ESRD: End-...,69 y/o female wtih PMH significant for ESRD on...
...,...,...,...,...
7338,Corrected(1835),"Azitromisin tedavisi sonlandırıldıktan sonra, ...",advers reaksiyon ve yan etki aynı anlamda kull...,NaN
7352,En(1839),He had been on a diazepam CIWA scale prophylax...,DTs: Delirium Tremens: Deliryum tremens.,He had been on a diazepam CIWA scale prophylax...
7392,En(1849),5) Anemia - previous w/u c/w ACD ( elev ferrit...,w/u: Workup: tetkikler; ACD: Anemia of chronic...,5) Anemia - previous w/u c/w ACD ( elev ferrit...
7512,En(1879),4. CAD s/p velocity stenting proximal LAD 2002,Velocity bir stent markası.,4. CAD s/p velocity stenting proximal LAD 2002


In [12]:
TRDatas = TRDatas.drop(columns=['Index', 'Content', 'Content.1'])
TRDatas


KeyError: "['Index', 'Content', 'Content.1'] not found in axis"

In [13]:
TRDatas.to_json("TRDatas.json", orient="records" indent=4)

# Veri Düzenleme
Daha önceden hazırlanmış çeviri verilerinin standart sağlamak amacıyla şu şekilde düzenleme yapılmıştır:
```
EN_KISALTMA:EN_AÇIKLAMA:TR_AÇIKLAMA:TR_KISALTMA
```

EN_AÇIKLAMA ve TR_AÇIKLAMA ya string oluyor ya da çoğul anlamdaysa [TR_AÇIKLAMA_1, TR_AÇIKLAMA_2] olarak çoğaltılıyor.
```

Eksik olan veriler boş bırakılıyor, yeni veri varsa ; işaretiyle devam ediyor. Örnek:

```json
        "12":"CHF:congestive heart failure:Konjestif kalp yetmezliği:KKY|OSH:outside hospital:dış merkez",
        "13":"Admission::[hastaneye yatış, hastaneye başvurmak]",
```

In [14]:
TRDatas = pd.read_json("TRDatas.json", orient="records")

In [15]:
TRDatas

,Notes
0,AV:arteriovenous:arteriyovenöz
12,CHF:congestive heart failure:Konjestif kalp ye...
13,"Admission::[""hastaneye yatış"", ""hastaneye başv..."
20,CPAP:Continuous Positive Airway Pressure:Devam...
24,PMH:Past Medical History:özgeçmiş; ESRD:End-St...
...,...
7338,advers reaksiyon ve yan etki aynı anlamda kull...
7352,DTs: Delirium Tremens: Deliryum tremens.
7392,w/u: Workup: tetkikler; ACD: Anemia of chronic...
7512,


In [23]:
results = []
for _, row in TRDatas.iterrows():
    note = row["Notes"].strip()
    
    if not note:
        continue  # Boş olanları atla

    # Kısaltma bloklarını ";" ile ayır
    blocks = note.split(";")

    for block in blocks:
        # Regex ile "EN_KISALTMA:EN_AÇIKLAMA:TR_AÇIKLAMA" desenini bul
        match = re.match(r"([^:]+):\s*([^:]+):\s*([\w\sğüşıöç,.-]*)", block.strip())
        if match:
            en_kisaltma, en_aciklama, tr_aciklama = match.groups()
            
            tr_aciklama_list = [x.strip() for x in tr_aciklama.split(",")] if "," in tr_aciklama else tr_aciklama.strip()

            results.append({
                "en": en_kisaltma,
                "en_text": en_aciklama,
                "tr_text": tr_aciklama_list
            })


In [24]:
results

[{'en': 'AV', 'en_text': 'arteriovenous', 'tr_text': 'arteriyovenöz'},
 {'en': 'CHF',
  'en_text': 'congestive heart failure',
  'tr_text': 'Konjestif kalp yetmezliği'},
 {'en': 'OSH', 'en_text': 'outside hospital', 'tr_text': 'dış merkez'},
 {'en': 'CPAP',
  'en_text': 'Continuous Positive Airway Pressure',
  'tr_text': 'Devamlı Pozitif Havayolu Basıncı'},
 {'en': 'PMH', 'en_text': 'Past Medical History', 'tr_text': 'özgeçmiş'},
 {'en': 'ESRD',
  'en_text': 'End-Stage Renal Disease',
  'tr_text': 'Son Dönem Böbrek Hastalıgı'},
 {'en': 'HD', 'en_text': 'hemodialysis', 'tr_text': 'hemodiyaliz'},
 {'en': 'Type 2 DM',
  'en_text': 'Type 2 Diabetes Mellitus',
  'tr_text': 'Tip 2 Diabetes Mellitus'},
 {'en': 'PE', 'en_text': 'Pulmonary Embolism', 'tr_text': 'Pulmoner Emboli'},
 {'en': 'PEA',
  'en_text': 'Pulseless Electrical Activity',
  'tr_text': 'nabızsız elektriksel aktivite'},
 {'en': 'EKG',
  'en_text': 'Elektrokardiyogram',
  'tr_text': 'Elektrokardiyogram'},
 {'en': 'UA', 'en_text'